In [1]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
asset = pdr.get_data_famafrench('5_Industry_Portfolios', start='2000', end='2020')
factor = pdr.get_data_famafrench('F-F_Research_Data_Factors', start='2000', end='2020')

asset = asset[0]/100
factor = factor[0]/100

In [2]:
asset.head(3)

,Cnsmr,Manuf,HiTec,Hlth,Other
Date,,,,,
2000-01,-0.0795,-0.0496,-0.0454,0.0754,-0.0469
2000-02,-0.0511,-0.0475,0.1329,-0.0292,-0.0712
2000-03,0.1103,0.0819,0.0463,0.0030,0.1408


In [3]:
factor.head(3)

,Mkt-RF,SMB,HML,RF
Date,,,,
2000-01,-0.0474,0.0579,-0.0189,0.0041
2000-02,0.0245,0.2148,-0.0981,0.0043
2000-03,0.0520,-0.1729,0.0823,0.0047


In [4]:
(1+asset).head(3) # 總報酬

,Cnsmr,Manuf,HiTec,Hlth,Other
Date,,,,,
2000-01,0.9205,0.9504,0.9546,1.0754,0.9531
2000-02,0.9489,0.9525,1.1329,0.9708,0.9288
2000-03,1.1103,1.0819,1.0463,1.0030,1.1408


In [10]:
egg = (1+asset)
egg2 = (1+asset).rolling(3).apply(np.prod)

In [13]:
print(egg.head(3))
egg2.head(3)

          Cnsmr   Manuf   HiTec   Hlth    Other
Date                                           
2000-01  0.9205  0.9504  0.9546  1.0754  0.9531
2000-02  0.9489  0.9525  1.1329  0.9708  0.9288
2000-03  1.1103  1.0819  1.0463  1.0030  1.1408


,Cnsmr,Manuf,HiTec,Hlth,Other
Date,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN
2000-03,0.969805,0.979396,1.131538,1.04713,1.009881


In [14]:
0.9205*0.9489*1.1103  

0.969805358235

In [ ]:
# MOM: take last 11 months' cumulative return
(1+asset).rolling(11).apply(np.prod).head(10)

之後的步驟跟 Lecture 11 相似

In [16]:
asset_d = pdr.get_data_famafrench('10_Industry_Portfolios_daily', start='2018', end='2020')
factor_d = pdr.get_data_famafrench('F-F_Research_Data_Factors_daily', start='2018', end='2020')
asset_d = asset_d[0]/100
factor_d = factor_d[0]/100

print(asset_d.head(5))
# resample daily to monthly
asset_d.resample("M").max().head()
print('------')
print(asset_d.head(5))

             NoDur   Durbl   Manuf   Enrgy   HiTec   Telcm   Shops   Hlth   \
Date                                                                         
2018-01-02 -0.0056  0.0159  0.0054  0.0167  0.0135  0.0153  0.0097  0.0140   
2018-01-03 -0.0027  0.0064  0.0039  0.0135  0.0114 -0.0093  0.0057  0.0089   
2018-01-04  0.0050  0.0124  0.0062  0.0042  0.0048  0.0020  0.0008 -0.0018   
2018-01-05  0.0028  0.0066  0.0082 -0.0006  0.0101  0.0029  0.0101  0.0059   
2018-01-08  0.0011  0.0088  0.0052  0.0053  0.0041 -0.0032  0.0048 -0.0048   

             Utils   Other  
Date                        
2018-01-02 -0.0039  0.0043  
2018-01-03 -0.0053  0.0052  
2018-01-04 -0.0061  0.0076  
2018-01-05 -0.0018  0.0059  
2018-01-08  0.0081 -0.0005  
------
             NoDur   Durbl   Manuf   Enrgy   HiTec   Telcm   Shops   Hlth   \
Date                                                                         
2018-01-02 -0.0056  0.0159  0.0054  0.0167  0.0135  0.0153  0.0097  0.0140   
2018-01-0

In [ ]:
start_time = time.time()
IVol = pd.DataFrame(index=asset_d.resample("M").last().index, columns=asset_d.
,→columns)
ISkew = pd.DataFrame(index=asset_d.resample("M").last().index, columns=asset_d.
,→columns)
for ym in tqdm(factor_d.index.strftime('%Y-%m').unique()): 
    # convert daily index YYYY-MM-DD into year-month format
X = factor_d.loc[ym,"Mkt-RF":"HML"] # we only need these data (independent variable)
X.insert(0, 'Const', 1.0) # include the alpha in the FF3 regression

for c in ER_d.columns: # calculate with different industries
Y = ER_d.loc[ym,c]
res = sm.OLS(Y,X).fit()
IVol.loc[ym,c] = np.sqrt(res.mse_resid)
ISkew.loc[ym,c] = res.resid.skew()
print("--- %s seconds ---" % (time.time() - start_time))
IVol.tail()

In [ ]:
# ck for rollingOLS
start_time = time.time()
Coskew = pd.DataFrame(index=asset.index, columns=asset.columns)
exog = factor[['Mkt-RF']]
exog.insert(0, 'Const', 1.0)
endog = asset.subtract(factor.RF, axis=0)
winsize = 60
for t in tqdm(range(winsize, len(exog)+1)):
X = exog.iloc[t-winsize:t,:]
Y = endog.iloc[t-winsize:t,:]
eps_m = X['Mkt-RF'].subtract(X['Mkt-RF'].mean())
for i in range(0, len(endog.columns)):
res = sm.OLS(Y.iloc[:,i],X).fit()
eps_i = res.resid
Coskew.iloc[t-1,i] = np.mean(eps_i * (eps_m**2)) / (np.sqrt(np.
,→mean(eps_i**2)) * np.mean(eps_m**2))
print("--- %s seconds ---" % (time.time() - start_time))
Coskew.tail()